In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [71]:
def read_and_preprocess(path):
    df = pd.read_csv(path)
    df.drop("Unnamed: 0",axis=1,inplace=True)
    df["Failure"] = 0
    df.timestamp = pd.to_datetime(df.timestamp)
    df = df.set_index("timestamp")
    df.loc["2020-04-18 00:00:00":"2020-04-18 23:59:00","Failure"] = 1
    df.loc["2020-05-29 23:30:00":"2020-05-30 06:00:00","Failure"] = 1
    df.loc["2020-06-05 10:00:00":"2020-06-07 14:30:00","Failure"] = 1
    df.loc["2020-07-15 14:30:00":"2020-07-15 19:00:00","Failure"] = 1
    print("Reading and preprocess is successful!")
    return df

In [72]:
data = read_and_preprocess("MetroPT3(AirCompressor).csv")

Reading and preprocess is successful!


In [78]:
data.Failure.value_counts()[1]*100/len(data)

1.974622729322297